In [3]:
import numpy as np
from catboost import CatBoostRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for catboost from https://files.pythonhosted.org/packages/e2/63/379617e3d982e8a66c9d66ebf4621d3357c7c18ad356473c335bffd5aba6/catboost-1.2.2-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/47.0 kB ? eta -:--:--
     ---------------- --------------------- 20.5/47.0 kB 640.0 kB/s eta 0:00:01
     --------------------------------- ---- 41.0/47.0 kB 487.6 kB/s eta 0:00:01
     -------------------------------------- 47.0/47.0 kB 391.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/101.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/101.0 MB 3.4 MB/s eta 0:00:30
   ---------------------------------------- 0.4/101.0 MB 5.1 MB/s eta 0:00:20
    --------------------------------------- 2.4/101.0 MB 21.8 MB/s eta 0:00:05
   -- ------------------------------------- 5.6/101.0 MB 35.6 MB/s eta 0:00:03
   --- --------

In [5]:
import pandas as pd

In [6]:
dataset=pd.read_csv("insurance_pre.csv")

In [7]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [8]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [9]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,False,True
1,18,33.770,1,1725.55230,True,False
2,28,33.000,3,4449.46200,True,False
3,33,22.705,0,21984.47061,True,False
4,32,28.880,0,3866.85520,True,False
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,True,False
1334,18,31.920,0,2205.98080,False,False
1335,18,36.850,0,1629.83350,False,False
1336,21,25.800,0,2007.94500,False,False


In [10]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [11]:
independent=dataset.iloc[:, [0, 1, 2, 4, 5]]  #independent would contain a DataFrame with all rows, 
#but only the columns specified by their integer positions: the first, second, third, fifth, and sixth columns of dataset.

In [12]:
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,False,True
1,18,33.770,1,True,False
2,28,33.000,3,True,False
3,33,22.705,0,True,False
4,32,28.880,0,True,False
...,...,...,...,...,...
1333,50,30.970,3,True,False
1334,18,31.920,0,False,False
1335,18,36.850,0,False,False
1336,21,25.800,0,False,False


In [13]:
dependent=dataset[['charges']]

In [14]:
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
import multiprocessing

num_cores = multiprocessing.cpu_count()
print("Number of CPU cores:", num_cores)

Number of CPU cores: 8


In [58]:
param_grid = {
     'iterations': [10,50,100],
    'learning_rate': [0.1, 0.5],
    'depth': [4, 8],
    'l2_leaf_reg': [3, 5]
}

In [59]:
grid = GridSearchCV(CatBoostRegressor(),param_grid ,cv=3, verbose=3, n_jobs=-1)
#grid = GridSearchCV(estimator=LGBMRegressor(), param_grid=param_grid, cv=3, verbose=3, n_jobs=-1)
grid.fit(independent,dependent)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


C:\AI\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
2 fits failed out of a total of 72.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "C:\AI\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Prabakaran J\AppData\Roaming\Python\Python311\site-packages\catboost\core.py", line 5703, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

0:	learn: 11171.6271343	total: 2.44ms	remaining: 120ms
1:	learn: 10326.4832268	total: 5.13ms	remaining: 123ms
2:	learn: 9560.4925202	total: 7.47ms	remaining: 117ms
3:	learn: 8891.2411954	total: 9.59ms	remaining: 110ms
4:	learn: 8308.2173970	total: 11.7ms	remaining: 105ms
5:	learn: 7796.7388679	total: 13.9ms	remaining: 102ms
6:	learn: 7388.2790010	total: 16.6ms	remaining: 102ms
7:	learn: 6976.9139732	total: 18.8ms	remaining: 98.6ms
8:	learn: 6609.0799781	total: 20.7ms	remaining: 94.5ms
9:	learn: 6311.5212071	total: 23ms	remaining: 91.8ms
10:	learn: 6083.9097064	total: 25.1ms	remaining: 88.9ms
11:	learn: 5883.9239703	total: 27.1ms	remaining: 85.9ms
12:	learn: 5683.7043431	total: 29.5ms	remaining: 84.1ms
13:	learn: 5501.7215108	total: 31.8ms	remaining: 81.8ms
14:	learn: 5340.4990943	total: 34.2ms	remaining: 79.8ms
15:	learn: 5198.0128291	total: 36.6ms	remaining: 77.8ms
16:	learn: 5088.5068832	total: 39.2ms	remaining: 76ms
17:	learn: 4994.0290586	total: 41.4ms	remaining: 73.6ms
18:	learn: 

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostRegressor object at 0x000002A732692A50>,
             n_jobs=-1,
             param_grid={'depth': [4, 8], 'iterations': [10, 50, 100],
                         'l2_leaf_reg': [3, 5], 'learning_rate': [0.1, 0.5]},
             verbose=3)

In [60]:
print(independent.columns)

Index(['age', 'bmi', 'children', 'sex_male', 'smoker_yes'], dtype='object')


In [61]:
re=grid.cv_results_
#print(re)
#Printing grid.cv_results_ will provide detailed information about the cross-validation results 
#for each combination of hyperparameters

In [62]:
print("The R2_score value for the best parameter{}:".format(grid.best_params_))

The R2_score value for the best parameter{'depth': 4, 'iterations': 50, 'l2_leaf_reg': 3, 'learning_rate': 0.1}:


In [63]:
print("Best R^2 score:", grid.best_score_)

Best R^2 score: 0.857319850442061


In [64]:
table=pd.DataFrame.from_dict(re)

In [39]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_depth,param_iterations,param_l2_leaf_reg,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,1.194424,0.032452,0.008158,0.005042,4,100,3,0.1,"{'depth': 4, 'iterations': 100, 'l2_leaf_reg':...",0.870703,0.860990,0.839731,0.857142,0.012934,1
1,1.164581,0.022282,0.012023,0.005801,4,100,3,0.5,"{'depth': 4, 'iterations': 100, 'l2_leaf_reg':...",0.843368,0.837053,0.815438,0.831953,0.011959,8
2,1.059430,0.134166,0.007426,0.001829,4,100,5,0.1,"{'depth': 4, 'iterations': 100, 'l2_leaf_reg':...",0.871161,0.859963,0.838341,0.856488,0.013622,2
3,0.920418,0.073060,0.003290,0.003143,4,100,5,0.5,"{'depth': 4, 'iterations': 100, 'l2_leaf_reg':...",0.851200,0.845464,0.822987,0.839884,0.012175,7
4,3.327647,0.041322,0.008239,0.001938,4,500,3,0.1,"{'depth': 4, 'iterations': 500, 'l2_leaf_reg':...",0.854379,0.846736,0.826213,0.842443,0.011893,6
5,3.232429,0.107213,0.014642,0.002425,4,500,3,0.5,"{'depth': 4, 'iterations': 500, 'l2_leaf_reg':...",0.813940,0.802942,0.794160,0.803681,0.008092,14
6,3.431806,0.247693,0.009120,0.005021,4,500,5,0.1,"{'depth': 4, 'iterations': 500, 'l2_leaf_reg':...",0.859957,0.848981,0.827179,0.845373,0.013623,5
7,3.671218,0.096522,0.009438,0.004684,4,500,5,0.5,"{'depth': 4, 'iterations': 500, 'l2_leaf_reg':...",0.820550,0.811293,0.797069,0.809637,0.009658,13
8,1.574461,0.069801,0.005075,0.002874,8,100,3,0.1,"{'depth': 8, 'iterations': 100, 'l2_leaf_reg':...",0.860468,0.851361,0.831535,0.847788,0.012079,4
9,1.670949,0.072541,0.009633,0.005727,8,100,3,0.5,"{'depth': 8, 'iterations': 100, 'l2_leaf_reg':...",0.829044,0.810208,0.800203,0.813151,0.011957,12


In [56]:
import pickle
#import CatBoostRegressor

# Trained model object (assuming it's already trained and stored in the 'regressor' variable)
regressor = CatBoostRegressor(iterations=50)
regressor.fit(independent,dependent)

# Save the trained model to a file
filename = "finalized_model_catboost.sav"
pickle.dump(regressor, open(filename, 'wb'))

Learning rate set to 0.489663
0:	learn: 7768.2355938	total: 2.66ms	remaining: 130ms
1:	learn: 5877.4615312	total: 6.08ms	remaining: 146ms
2:	learn: 4981.0556625	total: 8.64ms	remaining: 135ms
3:	learn: 4654.1331671	total: 11.2ms	remaining: 129ms
4:	learn: 4488.5649457	total: 14.8ms	remaining: 133ms
5:	learn: 4393.8647952	total: 18.2ms	remaining: 133ms
6:	learn: 4374.9001484	total: 20.1ms	remaining: 123ms
7:	learn: 4358.9835310	total: 23.5ms	remaining: 123ms
8:	learn: 4326.4486992	total: 26.9ms	remaining: 123ms
9:	learn: 4307.3483564	total: 30.7ms	remaining: 123ms
10:	learn: 4279.1148858	total: 34.4ms	remaining: 122ms
11:	learn: 4254.4991361	total: 38ms	remaining: 120ms
12:	learn: 4203.6624191	total: 41.6ms	remaining: 119ms
13:	learn: 4194.5985781	total: 45.1ms	remaining: 116ms
14:	learn: 4170.2543132	total: 48.9ms	remaining: 114ms
15:	learn: 4152.8597795	total: 52.4ms	remaining: 111ms
16:	learn: 4126.1368082	total: 55.9ms	remaining: 109ms
17:	learn: 4085.7191891	total: 59.6ms	remaining

In [57]:
import pickle
import pandas as pd

# Load the trained model
loaded_model = pickle.load(open("finalized_model_catboost.sav", 'rb'))

# Define the prediction input
prediction = [[30, 22, 1, 1, 0]]
prediction_df = pd.DataFrame(prediction, columns=['age', 'bmi', 'children', 'sex_male', 'smoker_yes'])

# Make the prediction
result = loaded_model.predict(prediction_df)
print(result)

[4839.91881135]
